# VIE Airport Türen

In [1]:
from pandas import read_excel, concat, to_numeric
from datetime import datetime as dt

def clean_df(df):
    d = df.dropna(axis=1, how="all")
    new_columns = []
    for col in d.columns:
        col = col.lower().replace(" ", "_").replace(".", " ").replace("ü", "ue")
        col = col.replace("ä", "ae").replace("ö", "oe").replace("ß", "ss")
        col = col.replace(":", "").replace("?", "")
        new_columns.append(col.strip())
    d.columns = new_columns
    return d

int_mapper = lambda x: int(x)
x_mapper = lambda x: True if x=="X" else False
object_floor_mapper = lambda x: x.split("_")[0]
date_mapper = lambda x: dt.strptime(x, "%d.%m.%y")
cad_number_mapper = lambda x: f"{x:2d}"

# mapper functions for levels/floors
filemaker_levels = {
    '0':"00", '- 1':"K1", '1':"01", '2': "02", '3':"03", '4':"04", '- 2':"K2", 'ZG':"ZG",
    'K':"K1", '5':"05", '6':"06", '8':"08", '7':"07", 'DG':"DG", 'ZG1':"ZG1", 'EG':"EG",
    '9':"09", '10':"10", 'ZG2':"ZG2", '21':"21", '19':"19", '20':"20", '18':"18", '17':"17",
    '13':"13", '15':"15", '22':"22", '11':"11", '16':"16", '14':"14", '12':"12", '-2':"K2", '-1':"K1"
}
filemaker_level_mapper = lambda x: filemaker_levels[x] if x in filemaker_levels.keys() else "99"

bst_levels = {"0": "EG", "1": "01", "2": "02", "DG": "DG"}
bst_level_mapper = lambda x: bst_levels[x] if x in bst_levels.keys() else "99"

ebene_mapper = lambda x: "K1" if x=="K 1" else x

## NPA manuelle Mitarbeiterliste

Bei der Liste „Durch Mitarbeiter aufgenommene Anlagenliste (Dateinamen „NPA_Tür Aufnahme Obj. 420“)“ ist der Feuerwiderstand jenes Element, welches als Grundlage für den Vergleich mit den anderen Listen herangezogen werden soll. Folgende relevanten Elemente müssen mit den anderen Listen verglichen und aufgelistet werden

* AKS-Nummer Plan (setzt sich zusammen aus Objekt/Ebene/Bauteil/Aks Plan) – Bsp. 103 00 A0220.01
* AKS-Nummer real (setzt sich zusammen aus Objekt/Ebene/Bauteil/Aks real) – Bsp. 103 00 0A0220.01
* Schlossernummer (Bsp. 103/0/1)
* Alte Türnummer (Bsp. BW102)
* Fluchttürnummer (Spalte Q) neu

In [2]:
file_name = "NPA_Tuer Aufnahme Obj420.xlsx"

npa_data = read_excel(
    io=f"data/{file_name}"
)

npa_data["Ebene"] = npa_data["Ebene"].map(ebene_mapper)

npa_data["origin"] = file_name
npa_data["aks_soll"] = npa_data["Objekt"].astype(str) + " " + npa_data["Ebene"].astype(str) + "" + npa_data["Bauteil"].astype(str) + "" + npa_data["AKS Plan"].astype(str)
npa_data["aks_ist"] = npa_data["Objekt"].astype(str) + " " + npa_data["Ebene"].astype(str) + "" + npa_data["Bauteil"].astype(str) + "" + npa_data["AKS real"].astype(str)

npa_data.head()

,Objekt,Ebene,Bauteil,AKS Plan,AKS real,alte Türnummer,Schlossernummer,Höhe [cm],Breite [cm],Zylinder [Z/B/WC/S],...,Panik [D/ St.K],Nottaster [ja/nein],Offenhalte-magnet [ja/nein],Taster für Schließung [ja/nein],Vorsicherung [ja/nein],SZ - Magnet [ja/nein],Bemerkung,origin,aks_soll,aks_ist
0,420,K1,A,2205.01,keine Nr.,5K2,063 / K / 48,200,80,nein,...,D / D,nein,ja,ja,nein,nein,im Plan rot,NPA_Tuer Aufnahme Obj420.xlsx,420 K1A2205.01,420 K1Akeine Nr.
1,420,K1,A,2506.01,keine Nr.,5K3,063 / K / 47,200,80,nein,...,D / D,nein,ja,ja,nein,nein,im Plan rot,NPA_Tuer Aufnahme Obj420.xlsx,420 K1A2506.01,420 K1Akeine Nr.
2,420,K1,A,2308.01,keine Nr.,01A,keine Nr.,243,112,nein,...,D / D,nein,nein,nein,nein,nein,NaN,NPA_Tuer Aufnahme Obj420.xlsx,420 K1A2308.01,420 K1Akeine Nr.
3,420,K1,A,2108.01,keine Nr.,5K1,063 / K / 109,200,80,nein,...,D / D,nein,ja,ja,nein,nein,im Plan rot,NPA_Tuer Aufnahme Obj420.xlsx,420 K1A2108.01,420 K1Akeine Nr.
4,420,K1,A,1305.02,keine Nr.,3K1,063 / K / 149,200,80,nein,...,D / D,nein,ja,ja,nein,nein,im Plan rot,NPA_Tuer Aufnahme Obj420.xlsx,420 K1A1305.02,420 K1Akeine Nr.


## Haltemagnete Schrack

In [3]:
file_name = "Schrack_HM_Obj420.xls"
columns = [
    "object",
    "level",
    "aks",
    "door_id_alt",
    "bst_door_id_alt",
    "room_id",
    "aks_room_id",
    "uz6_control",
    "door_series",
    "lfd_id",
    "info"
]

hm_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="HALTEMAGNETE"
)

hm_data.columns = columns
hm_data.dropna(inplace=True, how="all")

# Add origin
hm_data["origin"] = file_name

# hm_data["object"] = hm_data["object"].astype(int)
# hm_data["uz6_control"] = hm_data["uz6_control"].astype(int)
# hm_data["door_series"] = hm_data["door_series"].astype(int)
# hm_data["lfd_id"] = hm_data["lfd_id"].astype(int)

hm_data.head(5)

,object,level,aks,door_id_alt,bst_door_id_alt,room_id,aks_room_id,uz6_control,door_series,lfd_id,info,origin
0,420.0,EG,420 EG0A0106.02,004,420/0/3/6,001,0A0106,5.0,1.0,1.0,NaN,Schrack_HM_Obj420.xls
2,420.0,EG,420 EG0A0803.02,040D,420/0/14/6,040D,0A0803,6.0,2.0,2.0,NaN,Schrack_HM_Obj420.xls
3,420.0,EG,420 EG0A0602.01,040A,420/0/15/6,040A,0A0602,6.0,2.0,3.0,NaN,Schrack_HM_Obj420.xls
4,420.0,EG,420 EG0A0406.01,008,420/0/22/6,004,0A0406,6.0,2.0,4.0,NaN,Schrack_HM_Obj420.xls
5,420.0,EG,420 EG A0807.02,009,420/0/10,009,0A0807,6.0,2.0,5.0,NaN,Schrack_HM_Obj420.xls


## Brandschutztechnik (BST) Sisando

In [4]:
file_name = "Sisando_BST_Obj420.xlsx"

columns = [
    "object",
    "level",
    "door_id",
    "status_text",
    "status",
    "door_class",
    "room_id",
    "room_id_additional",
    "door_audit_remark",
    "door_audit_date",
    "door_audit_responsible_short",
    "door_audit_responsible_long",
    "electrical_operation",
    "exf",
    "retention_mechanism",
    "manufacturer",
    "classification"
]

bst_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="Sheet1",
    parse_dates=[9]
)

bst_data.columns = columns

bst_data["object"] = bst_data["object"].astype(str)
bst_data["origin"] = file_name
bst_data["aks"] = bst_data["object"].astype(str) + " " + bst_data["level"].map(ebene_mapper).astype(str) + bst_data["door_id"]

bst_data.head(5)

,object,level,door_id,status_text,status,door_class,room_id,room_id_additional,door_audit_remark,door_audit_date,door_audit_responsible_short,door_audit_responsible_long,electrical_operation,exf,retention_mechanism,manufacturer,classification,origin,aks
0,420,0,A0201.01,OK,1,Drehflügel #1,A0201,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,NaN,Peneder,T30,Sisando_BST_Obj420.xlsx,420 0A0201.01
1,420,0,A0203.01,OK,1,Drehflügel #2,A0203,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Peneder,EI2-30-C,Sisando_BST_Obj420.xlsx,420 0A0203.01
2,420,0,A0803.01,OK,1,Drehflügel #1,A0803,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Peneder,EI2-30-C,Sisando_BST_Obj420.xlsx,420 0A0803.01
3,420,0,A0803.02,HINWEIS,21,Drehflügel #2,A0803,NaN,STATUS OK_x001D_H000 Freilauftürschließer hält...,2024-10-28,peD,David PESCHEK,NaN,1.0,NaN,Fritscher,T30,Sisando_BST_Obj420.xlsx,420 0A0803.02
4,420,0,A0807.03,OK,1,Drehflügel #2,A0807,NaN,STATUS OK_x001D__x001D__x001D_,2024-10-28,peD,David PESCHEK,NaN,NaN,NaN,Fritscher,T30,Sisando_BST_Obj420.xlsx,420 0A0807.03


## Feuerleittechnik (FLT) Sisando

In [5]:
file_name = "Sisando_FLT_Obj420.xlsx"

columns = [
    "row_id",
    "object",
    "module",
    "section",
    "floor",
    "room_id",
    "number",
    "is_emergency_door",
    "is_emergency_door_no",
    "has_blind_cylinder",
    "has_cylinder",
    "has_panic_lock",
    "is_locked",
    "is_lockable",
    "is_not_lockable",
    "has_emergency_button",
    "has_emergency_button_no",
    "defect_memo",
    "defect_fixed",
    "none_a",
    "none_b",
    "none_c"
]

flt_data = read_excel(
    io=f"data/{file_name}",
    sheet_name="all",
    skiprows=[0,1],
    header=None
)

flt_data.columns = columns

flt_data.dropna(axis=1, how="all", inplace=True)

# Deleting of unused rows
flt_data["has_emergency_button"] = flt_data["has_emergency_button"].map(x_mapper)
flt_data["is_emergency_door"] = flt_data["is_emergency_door"].map(x_mapper)
flt_data["has_blind_cylinder"] = flt_data["has_blind_cylinder"].map(x_mapper)
flt_data["has_cylinder"] = flt_data["has_cylinder"].map(x_mapper)
flt_data["has_panic_lock"] = flt_data["has_panic_lock"].map(x_mapper)
flt_data["is_lockable"] = flt_data["is_lockable"].map(x_mapper)
flt_data["has_panic_lock"] = flt_data["has_panic_lock"].map(x_mapper)
flt_data["object"] = flt_data["object"].map(object_floor_mapper)
flt_data.drop("row_id", axis=1, inplace=True)
flt_data.drop("has_emergency_button_no", axis=1, inplace=True)

# Add origin
flt_data["origin"] = file_name
flt_data["aks"] = flt_data["object"].astype(str) + " " + flt_data["section"].astype(str) + flt_data["module"].astype(str) + flt_data["room_id"].astype(str) + "." + flt_data["number"].astype(str)

flt_data.head(5)

,object,module,section,floor,room_id,number,is_emergency_door,is_emergency_door_no,has_blind_cylinder,has_cylinder,has_panic_lock,is_lockable,is_not_lockable,has_emergency_button,defect_memo,origin,aks
0,420,A,EG,A,3108,01,True,NaN,False,True,False,True,NaN,False,NaN,Sisando_FLT_Obj420.xlsx,420 EGA3108.01
1,420,A,EG,NaN,5206,05,False,X,False,True,False,True,NaN,False,"Panikschloss fehlt (Außentür, Offline-Beschlag?)",Sisando_FLT_Obj420.xlsx,420 EGA5206.05
2,420,A,EG,NaN,4401,07,True,NaN,False,False,False,False,X,False,NaN,Sisando_FLT_Obj420.xlsx,420 EGA4401.07
3,420,A,EG,NaN,2601,06,True,NaN,False,True,False,True,NaN,False,NaN,Sisando_FLT_Obj420.xlsx,420 EGA2601.06
4,420,A,EG,NaN,2601,04,True,NaN,False,True,False,True,NaN,False,NaN,Sisando_FLT_Obj420.xlsx,420 EGA2601.04


## CAD Database

* Die Spalten B bis F ergeben die AKS-Nummer (``GAR-TURNUMMER-BAUTEIL``, ``GAR-TURNUMMER-EBENE``, ``GAR-TURNUMMER-MODUL``, ``GAR-TURNUMMER-AKS-NR``, ``GAR-TURNUMMER-NUMMER``)
* Fluchttürnummer in Spalte H (``GAR-FLUCHTTUERNUMMER``)

In [6]:
file_name = "134_gesamt_20241212.xlsx"

cad_data = read_excel(
    io = f"data/{file_name}",
    sheet_name="134_TÜRSTEMPEL"
)

cad_data["origin"] = file_name
cad_data["GAR-TURNUMMER-NUMMER"] = cad_data["GAR-TURNUMMER-NUMMER"].map(cad_number_mapper)
cad_data["aks"] = cad_data["GAR-TURNUMMER-BAUTEIL"].astype(str) + " " + cad_data["GAR-TURNUMMER-EBENE"].astype(str) + cad_data["GAR-TURNUMMER-MODUL"].astype(str) + cad_data["GAR-TURNUMMER-AKS-NR"].astype(str) + "." + cad_data["GAR-TURNUMMER-NUMMER"].astype(str)


cad_data.head()

,Name,GAR-TURNUMMER-BAUTEIL,GAR-TURNUMMER-EBENE,GAR-TURNUMMER-MODUL,GAR-TURNUMMER-AKS-NR,GAR-TURNUMMER-NUMMER,GAR-TURNUMMER-ALT,GAR-FLUCHTTUERNUMMER,GAR-TUR-BREITE,GAR-TUR-HOHE,GAR-BSK,GAR-HEL,GAR-SEC,GAR-ZUSATZ-1,origin,aks
0,GAR-61-Turnummer-2013,134,00,A,641,1,005A,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A641. 1
1,GAR-61-Turnummer-2013,134,00,A,710,1,---,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A710. 1
2,GAR-61-Turnummer-2013,134,00,A,711,1,---,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A711. 1
3,GAR-61-Turnummer-2013,134,00,A,713,1,---,NaN,NaN,NaN,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A713. 1
4,GAR-61-Turnummer-2013,134,00,A,722,1,0A0722.01,NaN,85,210,NEIN,---,---,---,134_gesamt_20241212.xlsx,134 00A722. 1


In [10]:
cad_data["GAR-TURNUMMER-EBENE"].value_counts()

GAR-TURNUMMER-EBENE
00    134
K1     83
03     26
02     26
08     26
07     23
04     23
06     23
05     21
01     17
DA      1
Da      1
Name: count, dtype: int64

In [11]:
cad_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Name                   404 non-null    object
 1   GAR-TURNUMMER-BAUTEIL  404 non-null    int64 
 2   GAR-TURNUMMER-EBENE    404 non-null    object
 3   GAR-TURNUMMER-MODUL    404 non-null    object
 4   GAR-TURNUMMER-AKS-NR   404 non-null    int64 
 5   GAR-TURNUMMER-NUMMER   404 non-null    object
 6   GAR-TURNUMMER-ALT      404 non-null    object
 7   GAR-FLUCHTTUERNUMMER   93 non-null     object
 8   GAR-TUR-BREITE         292 non-null    object
 9   GAR-TUR-HOHE           292 non-null    object
 10  GAR-BSK                404 non-null    object
 11  GAR-HEL                404 non-null    object
 12  GAR-SEC                404 non-null    object
 13  GAR-ZUSATZ-1           404 non-null    object
 14  origin                 404 non-null    object
 15  aks                    

## Filemaker Database

In [7]:
file_name = "BSTüren_Filemaker_11.04.22.xlsx"

filemaker_data = clean_df(
    read_excel(
        io=f"data/{file_name}",
        sheet_name="Gesamt"
    )
)

# We only use object 420
filemaker_data = filemaker_data.loc[filemaker_data["bauteil"]=="420"]

filemaker_data["ebene"] = filemaker_data["ebene"].map(filemaker_level_mapper)
filemaker_data["origin"] = file_name
filemaker_data["aks"] = filemaker_data["bauteil"].astype(str) + " " + filemaker_data["brandmeldernr"].astype(str)

filemaker_data.head()

,bauteil,ebene,topnr,brandmelderkette,brandmeldernr,erfasst_von,tuer_12flg,zugaengigkeit,ok,hersteller,...,tax,gesuanbot,arbeitszeit,material,schadensmeldung,aufnahme,kleinregie_summe,zyklus,origin,aks
4594,420,00,78.0,021,B2605.01,Block,1,NaN,2.0,FRITSCHER,...,1130335.5,6782013,NaN,NaN,NaN,744,10,04,BSTüren_Filemaker_11.04.22.xlsx,420 B2605.01
4595,420,00,79.0,018A,B1607.03,Block,1,NaN,2.0,FRITSCHER,...,1130335.5,6782013,NaN,NaN,NaN,744,10,04,BSTüren_Filemaker_11.04.22.xlsx,420 B1607.03
4596,420,00,80.0,006,B1607.01,Block,2 FLG. Li,NaN,2.0,FRITSCHER,...,1130335.5,6782013,NaN,NaN,Quellstreifen bei allen alten Türen die neu la...,744,10,04,BSTüren_Filemaker_11.04.22.xlsx,420 B1607.01
4597,420,00,81.0,\nBüro,B1605.02,Block,1 FLG. Re,NaN,1.0,NOVOFERM,...,1130335.5,6782013,NaN,NaN,NaN,744,10,04,BSTüren_Filemaker_11.04.22.xlsx,420 B1605.02
4598,420,00,82.0,NaN,012,Block,1 FLG. Li,Kollektor,2.0,FRITSCHER,...,1130335.5,6782013,NaN,NaN,NaN,744,10,04,BSTüren_Filemaker_11.04.22.xlsx,420 012


## Integration of data